In [1]:
import time
import os
from os import listdir
from os.path import join, isfile
import glob
import numpy as np
import numpy.matlib
import pandas as pd
import pyedflib
from scipy.signal import find_peaks_cwt, welch
from scipy.signal import decimate
from datetime import timedelta
from datetime import datetime

from mne.filter import filter_data

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import roc_auc_score, roc_curve

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, hilbert, chirp
from scipy import signal
from scipy.fft import fft, ifft

In [2]:
p_right_anno = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/TM-metadata/TM-R'
p_left_anno = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/TM-metadata/TM-L'

In [3]:
p_right_data = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/TM-metadata/TM-R'
p_left_data = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/TM-metadata/TM-L'

In [4]:
right = pd.read_csv(p_right_anno + '/RECORDING_00.csv')['Time (us)']
left = pd.read_csv(p_left_anno + '/RECORDING_01.csv')['Time (us)']

### Right and left side time start and end

In [5]:
right

0            Electronics ID: ps-v1.3-000010
1       Log start time: 10:52:58 19-10-2021
2                         Medication Times:
3                                     20836
4                                     37575
5                                     69602
6          Log end time: 8:17:58 20-10-2021
7    Shut down due to: Electrode disconnect
Name: Time (us), dtype: object

In [6]:
left

0            Electronics ID: ps-v1.3-000002
1       Log start time: 10:59:28 19-10-2021
2                         Medication Times:
3                                     20427
4                                     37165
5                                     69191
6          Log end time: 8:17:59 20-10-2021
7    Shut down due to: Electrode disconnect
Name: Time (us), dtype: object

In [10]:
import datetime

time_left = datetime.datetime.strptime('10:59:28 19-10-2021', '%H:%M:%S %d-%m-%Y')
time_right = datetime.datetime.strptime('10:52:58 19-10-2021', '%H:%M:%S %d-%m-%Y')
seconds_in_day = 24 * 60 * 60
difference = time_left - time_right
time_div = divmod(difference.days * seconds_in_day + difference.seconds, 60)
print('The difference is',time_div[0], 'minutes and' ,time_div[1], ' seconds')
print(difference)

The difference is 6 minutes and 30  seconds
0:06:30


In [82]:
time_left_end = datetime.datetime.strptime('8:17:58 20-10-2021', '%H:%M:%S %d-%m-%Y')
time_right_end = datetime.datetime.strptime('8:17:59 20-10-2021', '%H:%M:%S %d-%m-%Y')
seconds_in_day = 24 * 60 * 60
difference = time_right_end - time_left_end  
time_div = divmod(difference.days * seconds_in_day + difference.seconds, 60)
print('The difference is',time_div[0], 'minutes and' ,time_div[1], ' seconds')
print(difference)

The difference is 0 minutes and 1  seconds
0:00:01


In [54]:
sampling_frequency = 500 

## Match two datasets from right and left

In [11]:
Left_paths = glob.glob("/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/L/RECORDING_01*.csv")
Right_paths = glob.glob("/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/R/RECORDING_00*.csv")

In [12]:
print('number of files left side', len(Left_paths))
print('number of files right side', len(Right_paths))

number of files left side 43
number of files right side 43


In [15]:
# For each file give correct time 
dfNames = ['Time','EMG','batt','gX','gY','gZ','aX','aY','aZ'] 

In [16]:
Left_paths[0]

'/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/L/RECORDING_01-OUT16.csv'

## Downsampling

In [67]:
# For each of the files divide into 90000 samples 
def downsampling(samplingrateBefore, samplingrateAfter, Right_paths,Left_paths):

    shapesR = []
    shapesL = []
    for paths in Left_paths: 
        data = pd.read_csv(paths)
        fL = signal.resample(data,round((samplingrateAfter/samplingrateBefore) * data.shape[0]))
        shapesL.append(fL.shape[0]
        # save to path left
        np.save('/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/Downsampling/L/'+paths[-9:-4]+'.npy',fL) 


    for paths in Right_paths: 
        data = pd.read_csv(paths)
        fR = signal.resample(data,round((samplingrateAfter/samplingrateBefore) * data.shape[0]))
        shapesR.append(fR.shape[0])
        # save to path right
        np.save('/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/Downsampling/R/'+paths[-9:-4]+'.npy',fR) 
    return shapesR,shapesL


In [78]:
# downsampling(2000, 100, Right_paths,Left_paths) 
print('Mismatch in total size')
(np.sum(shapesR)-np.sum(shapesL))/(100*60)

mismatch in total size


6.476666666666667

## Merge according the last file towards the first


In [83]:
np.sum(shapesR)-np.sum(shapesL)

38860

In [103]:
# as we have downsampled, the computer will be able to handel the complete data easier 
left_data = []
right_data = []

path_r = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/Downsampling/R/'
path_l = '/Users/jiayihan/OneDrive - Danmarks Tekniske Universitet/Thesis/paragitData/Downsampling/L/'

for file in range(43):
    filename = 'OUT'+str(file).zfill(2)+'.npy' 
    left_data.append(np.load(path_l+filename).astype(np.float64))
    right_data.append(np.load(path_r+filename).astype(np.float64))


In [144]:
left_complete = np.concatenate(left_data)    
right_complete = np.concatenate(right_data)

In [150]:
cut_tail_left = left_complete[:-140,:]
cut_front_right = right_complete[30*100+(6*60*100):,:]

In [152]:
dfNames = ['Time','EMG','batt','gX','gY','gZ','aX','aY','aZ'] # Data columns in csv file

In [155]:
Left = pd.DataFrame(data=cut_tail_left)
Right = pd.DataFrame(data = cut_front_right)

In [157]:
Right.columns = dfNames
Left.columns = dfNames

In [159]:
Right[['aX','aY','aZ','gX','gY','gZ']] 

,aX,aY,aZ
0,-1177.302969,12601.079750,10816.128563
1,-1251.278514,12749.065569,10793.318951
2,-1141.182686,12680.527765,10739.373551
3,-977.084162,12449.804436,10662.235127
4,-838.295793,12121.904824,10708.114616
...,...,...,...
7670316,288.678659,10688.214967,13572.660520
7670317,198.824544,9752.550094,12978.875827
7670318,1251.905562,9666.386188,13871.610526
7670319,1396.867681,8661.765648,11792.521014


In [160]:
Left[['aX','aY','aZ','gX','gY','gZ']] 

,aX,aY,aZ,gX,gY,gZ
0,-327.413887,-13504.049953,6529.079764,-0.136883,-71.989311,-9.353248
1,4941.384612,-11268.250156,11934.321024,38.410029,22.187801,2.994291
2,3641.011171,-11696.037716,10609.905233,26.621994,-1.675077,-3.370447
3,4406.232679,-11317.273458,11331.681110,28.724171,6.109321,-4.310424
4,3867.966401,-11430.841427,10765.615639,25.646787,-4.107055,-6.653825
...,...,...,...,...,...,...
7670316,-1947.219371,-6290.409692,14859.560044,-128.156828,-881.289010,-327.741816
7670317,-1437.462498,-7873.796836,14145.107683,221.172719,-890.934394,-355.036181
7670318,-670.220870,-8664.367929,13849.799759,166.402969,-674.334781,-419.320113
7670319,-502.559918,-8537.099957,13400.581755,13.182115,-333.486889,-517.473159


In [175]:
# RightLeft = pd.concat([Right[['aX','aY','aZ','gX','gY','gZ']] , Left[['aX','aY','aZ','gX','gY','gZ']] ], axis=1)
#numpyRightLeft=RightLeft.to_numpy()
#np.save('numpyRightLeft.npy',numpyRightLeft)
#np.load('numpyRightLeft.npy').astype(np.float64).shape